In [9]:
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

SURVEY_SUMMARY_PATH = r'E:\Met_Council\metc-asim-model\Input\Visualizer\base'
ASIM_OUTPUT = r"E:\Met_Council\metc-asim-model\Base_2018\activitysim\output"
ASIM_CONFIGS_PATH = r'E:\Met_Council\metc-asim-model\source\activitysim\configs'
CENSUS_SUMMARY_PATH = r'E:\Met_Council\metc-asim-model\Input\Visualizer\census'

DAMPING = 0.75

coefficient_dict = {
    0: 'asc_allhhs_0_autos',
    1: None,
    2: 'asc_allhhs_2_autos',
    3: 'asc_allhs_3_autos',
    4: 'asc_allhhs_4plus_autos'
}

observed_auto_ownership = pd.read_csv(os.path.join(CENSUS_SUMMARY_PATH, "autoOwnershipCensus_2021.csv"))

observed_auto_ownership = observed_auto_ownership.rename(columns = {'HHVEH': 'auto_ownership'}).groupby('auto_ownership').agg(census_pct = ('freq', 'sum')) / observed_auto_ownership['freq'].sum()

model_hh = pd.read_csv(os.path.join(ASIM_OUTPUT, 'final_households.csv'))
model_hh['num_adults'] = model_hh['num_adults'].clip(0, 4)
model_data = model_hh.groupby('auto_ownership').agg(model_hh = ('household_id', 'count'))
model_data['model_pct'] = model_data['model_hh'] / model_data['model_hh'].sum()

coefficients = pd.read_csv(os.path.join(ASIM_CONFIGS_PATH, 'auto_ownership_coefficients.csv'))

In [10]:
compare_table = observed_auto_ownership.join(model_data)
compare_table['coefficient_name'] = ""
for i, v in compare_table.iterrows():
    if coefficient_dict[i] != None:
        compare_table.loc[i, 'coefficient_name'] = coefficient_dict[i]
compare_table = compare_table.merge(coefficients, how = 'left', on = 'coefficient_name')
compare_table['adj'] = np.log(compare_table['census_pct'] / compare_table['model_pct']) * DAMPING
compare_table.loc[compare_table['constrain'] != 'F', 'adj'] = 0.0
compare_table.rename(columns = {'value': 'old_value'}, inplace = True)
compare_table['value'] = compare_table['old_value'].astype(float).fillna(0) + compare_table['adj'].fillna(0)
compare_table

,census_pct,model_hh,model_pct,coefficient_name,old_value,constrain,adj,value
0,0.066244,20953,0.069843,asc_allhhs_0_autos,0.768233,F,-0.039682,0.728551
1,0.304022,84213,0.280710,,NaN,NaN,0.000000,0.000000
2,0.407081,125782,0.419273,asc_allhhs_2_autos,0.669453,F,-0.022133,0.647321
3,0.153699,47725,0.159083,asc_allhs_3_autos,0.719627,F,-0.025826,0.693800
4,0.068955,21327,0.071090,asc_allhhs_4plus_autos,-0.078699,F,-0.022875,-0.101574


In [11]:
compare_table[compare_table['coefficient_name'].str.len() > 0][['coefficient_name', 'value', 'constrain']].to_csv(sys.stdout, index = False)

coefficient_name,value,constrain
asc_allhhs_0_autos,0.7285506739764057,F
asc_allhhs_2_autos,0.6473207794475156,F
asc_allhs_3_autos,0.693800382727227,F
asc_allhhs_4plus_autos,-0.10157416335388413,F
